## 1. 라이브러리 설치

In [1]:
!pip install pdfplumber

## 2. Bedrock 설정 및 호출 함수 정의

In [2]:
import json
import boto3
from pprint import pprint

# Bedrock runtime 생성
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

In [3]:
def invoke_bedrock(
    query: str,
    model_id: str = "anthropic.claude-3-sonnet-20240229-v1:0",
    max_tokens: int = 4096,
    temperature: float = 1.0,
    top_p: float = 1.0,
    top_k: int = 250,
) -> str:

    # Request body 작성
    request_data = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                ],
            }
        ],
    }
    request_body = json.dumps(request_data)

    # 모델 호출
    response = bedrock_runtime.invoke_model(
        body=request_body,
        modelId=model_id,
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    answer = response_body['content'][0]['text']
    return answer

## 3. PDF 문서 읽는 함수 정의¶

In [4]:
import pdfplumber

def get_texts_from_pdf(filepath: str, start_page:int, page_number:int):
    with pdfplumber.open(filepath) as pdf:
        number_of_pages = len(pdf.pages)
        end_page = start_page + page_number
        if end_page > number_of_pages:
            end_page = number_of_pages
            
        contents = ""
        for i in range(start_page, end_page):
            page = pdf.pages[i]
            contents += page.extract_text()
            contents += "\n\n"
        return contents

## 5. 챕터별로 요약하기

In [5]:
content = get_texts_from_pdf(filepath="Ferrous Markets Analytics Monthly_26 Jun 2024.pdf", start_page=0, page_number=99)
prompt = f"""

당신은 PDF 문서를 챕터별로 요약하는 AI 입니다.
주어진 <content> 태그 안에 있는 내용을 챕터 별로 요약해주세요.
각 챕터별로 전체적인 요약이 되어야 합니다.

<content>
{content}
</content>

<mandatory>
1. 숫자가 포함된 내용
</mandatory>

"""
result = invoke_bedrock(query=prompt, temperature=0, top_p=0)
print(result)

챕터별 요약과 필수 숫자 내용은 다음과 같습니다.

유럽 철강:
- 독일 HRC 가격은 2024년 634-690유로/mt, 2025년 677-733유로/mt로 전망됨
- 독일 제조업 PMI는 2024년 41.9-43.9, 2025년 43.5-44.4 전망
<mandatory>숫자 포함 내용: 634, 690, 677, 733, 41.9, 43.9, 43.5, 44.4</mandatory>

미국 철강: 
- 미국 HRC 가격은 2024년 $735-870/st, 2025년 $863-937/st로 전망됨  
- 미국 제조업 PMI는 2024년 48.7-49.9, 2025년 49.5-50.3으로 전망
<mandatory>숫자 포함 내용: 735, 870, 863, 937, 48.7, 49.9, 49.5, 50.3</mandatory>

중국 철강:
- 2024년 중국 철강 소비는 883백만 톤으로 전년 대비 1.8% 감소 전망
- 부문별로는 부동산 247백만 톤(-8.2%), 인프라 233백만 톤(+1.1%) 등
<mandatory>숫자 포함 내용: 883, 1.8, 247, 8.2, 233, 1.1</mandatory>

철광석:
- 구체적인 숫자 데이터는 없음

제철용 원료탄:  
- 구체적인 숫자 데이터는 없음  

철스크랩:
- 구체적인 숫자 데이터는 없음

에너지:
- 구체적인 숫자 데이터는 없음
